In [1]:
#importanting the necessary libraries
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pickle
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
import spotipy

In [2]:
#oppening my popvortex 100 hot songs file
popv_hot_100 = pd.read_csv('top_songs.csv')
popv_hot_100.head()

,song_title,artist_name
0,rockstar,dolly parton
1,new blue sun,andré 3000
2,higher,chris stapleton
3,cosmo,ozuna
4,poet’s resumé - ep,tim mcgraw


In [3]:
#oppening my spotify list
spotify_list = pd.read_csv('spotify_df_w_clusters.csv')
spotify_list.head()

,track.name,name,cluster
0,2K,Nosaj Thing,8
1,4 Billion Souls,The Doors,7
2,4 Minute Warning,Radiohead,2
3,7 Element,Vitas,0
4,#9 Dream,R.E.M.,6


In [4]:
# Stablishing  spotify connection
def open_spotify_connection():
    secrets_file = open("Secrets","r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    return sp;

In [5]:
# Song recommendation model
def song_recommender(title):
    # Convert the user input and song names to lowercase for case-insensitive comparison
    user_input_song_lower = user_input_song.lower()
    top_chart_songs_lower = popv_hot_100.apply(lambda x: x.str.lower())
    # Check if the user-inputted song is in the top chart songs list (case-insensitive)
    if user_input_song_lower in top_chart_songs_lower['song_title'].values:
        # Filter out the user-inputted song from the DataFrame
        top_chart_songs_without_input = popv_hot_100[
            top_chart_songs_lower['song_title'] != user_input_song_lower]
        # Recommend a random song from the filtered DataFrame
        recommended_song = top_chart_songs_without_input.sample().squeeze()
        print(f"You might also like: {recommended_song['song_title']} by {recommended_song['artist_name']}")
    else:
        #open spotify connection
        sp = open_spotify_connection()
        # Query Spotify
        results = sp.search(q=f"track:{user_input_song_lower}", type="track", limit=1)
        # Check if the query returns results (i.e., total > 0)
        if results['tracks']['total'] != 0:
            track_id = results['tracks']['items'][0]['id']
            #print(track_id)
            new_data = sp.audio_features(track_id)
            data_df = json_normalize(new_data[0])
            num_data_df = data_df.select_dtypes(include='number')
            # Use pickle to get the existing scaler and kmeans 
            kmeans = pickle.load(open('kmeans.pkl', 'rb'))
            scaler = pickle.load(open('scaler.pkl', 'rb'))
            scaler = scaler.transform(num_data_df)
            predicted_cluster = kmeans.predict(scaler)
            cluster = predicted_cluster[0]
            #print (cluster)
            rec_song = spotify_list[spotify_list['cluster'] == cluster].sample().squeeze()
            print(f"You might also like: {rec_song['track.name']} by {rec_song['name']}")
        else:
            print("The song is not in Spotify. Please try again")


In [6]:
# Get User Input
user_input_song = input("Enter a song: ")
song_recommender(user_input_song)

Enter a song: Beacuse I'happy
You might also like: Want To by Dua Lipa
